## Initial setup

In [ ]:
import pandas as pd
from load_dataset import download_from_kaggle, load_from_database

# df = download_from_kaggle()
df = load_from_database("HotelBooking")
df.head()

### Look at the raw dataset

In [ ]:
df.describe()

In [ ]:
df.info()

# Data Cleaning
### Missing values

In [ ]:
def info_na():
    missing_values_count = df.isnull().sum()
    if missing_values_count.sum() == 0:
        print("no missing data")

    else:
        missing_data = pd.DataFrame({
            'missing values': missing_values_count[missing_values_count > 0],
            'total entries': len(df),
        })
        missing_data['percentage'] = round((missing_data['missing values'] / missing_data['total entries']) * 100, 4)
    
        print(missing_data, '\n')
        
    full_data_row_count = df.dropna(axis=0)
    print(f"full data row count {full_data_row_count.shape[0]} / {df.shape[0]}") 
        
    full_data_column_count = df.dropna(axis=1)
    print(f"full data column count {full_data_column_count.shape[1]} / {df.shape[1]}")
    


In [ ]:
info_na()

In [ ]:
# for children column we can assume that if there is no data, there were no children
df['children'] = df['children'].fillna(0)

# safer would be to set the values as unknown
df['country'] = df['country'].fillna('Unknown')

# this column contains id of an agency and will most likely not be needed for future
# analysis, but can be worth keeping for now. fill it with 0 as "unknown" for now
df['agent'] = df['agent'].fillna(0)

# delete company column because 94% data is missing
df.drop(labels='company', axis=1, inplace=True)

In [ ]:
info_na()

In [ ]:
df.describe()

## Duplicates

In [ ]:
def info_duplicates():
    duplicates = df[df.duplicated()]
    print(len(duplicates), ' duplicated rows')
    return duplicates

In [ ]:
info_duplicates()

In [ ]:
# drop duplicates, keeping the first occurrence
df.drop_duplicates(inplace=True)
info_duplicates()

## Data Type Correction

In [ ]:
df.dtypes

In [ ]:
# convert arrival date year, month and day of month into a datetime
df = df.rename(columns={
    'arrival_date_year': 'year',
    'arrival_date_month': 'month',
    'arrival_date_day_of_month': 'day'
})
# months are written verbally, convert January to 1, February to 2 etc.
df['month'] = pd.to_datetime(df['month'], format='%B').dt.month
# aggregate all values into one column representing arrival date
df['arrival_date'] = pd.to_datetime(df[['year', 'month', 'day']])
# drop year month day and week, they are deprecated now
df.drop(columns={'year', 'month', 'day', 'arrival_date_week_number'}, inplace=True)

dtype_mapping = {
    'hotel' : 'category',
    'is_canceled' : 'bool',
    'lead_time' : 'int',
    'adults': 'int',
    'children': 'int',
    'babies': 'int',
    'meal': 'category',
    'country': 'category',
    'market_segment': 'category',
    'distribution_channel': 'category',
    'is_repeated_guest': 'bool',
    'previous_cancellations': 'int',
    'previous_bookings_not_canceled': 'int',
    'reserved_room_type': 'category',
    'assigned_room_type': 'category',
    'booking_changes': 'int',
    'deposit_type': 'category',
    'agent': 'int',
    'days_in_waiting_list': 'int',
    'customer_type': 'category',
    'adr': 'float',
    'required_car_parking_spaces': 'int',
    'total_of_special_requests': 'int',
    'reservation_status': 'category',
    'reservation_status_date': 'datetime64[ns]'
}

# correct datatype if it is different
for column, dtype in dtype_mapping.items():
    if df[column].dtype == dtype:
        continue
    if dtype == 'datetime64[ns]':
        df[column] = pd.to_datetime(df[column])
    else:
        df[column] = df[column].astype(dtype)


In [ ]:
df.dtypes 

In [ ]:
# check if all values were converted
info_na()

## Outliers

In [ ]:
def delete_outliers_and_normalise(column_name):
    fix_outliers(column_name)
    normalize(column_name)

def delete_outliers_and_standardize(column_name):
    fix_outliers(column_name)
    standardize(column_name)

def standardize(column_name):
    mean = df[column_name].mean()
    std = df[column_name].std()
    
    df[column_name] = (df[column_name] - mean) / std

def normalize(column_name):
    min_val = df[column_name].min()
    max_val = df[column_name].max()

    if min_val != max_val:
        df[column_name] = (df[column_name] - min_val) / (max_val - min_val)

def fix_outliers(column_name, method="iqr"):

    if method.lower() == "iqr":
        Q1 = df[column_name].quantile(0.25)
        Q3 = df[column_name].quantile(0.75)
        IQR = Q3 - Q1
    
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
    else:
        mean = df[column_name].mean()
        std_dev = df[column_name].std()
        std_dev_multiplier = 4
        lower_bound = mean - std_dev_multiplier * std_dev
        upper_bound = mean + std_dev_multiplier * std_dev
    
    df.loc[(df[column_name] < lower_bound), column_name] = lower_bound.astype(df[column_name].dtype)
    df.loc[(df[column_name] > upper_bound), column_name] = upper_bound.astype(df[column_name].dtype)

In [ ]:
df.describe()

In [ ]:
fix_outliers('lead_time', method='std')
fix_outliers('stays_in_weekend_nights', method='iqr')
fix_outliers('stays_in_week_nights', method='iqr')
fix_outliers('adults', method='std')
fix_outliers('children', method='std')
fix_outliers('previous_cancellations', method='std')
fix_outliers('previous_bookings_not_canceled', method='std')
fix_outliers('booking_changes', method='std')
fix_outliers('days_in_waiting_list', method='std')
fix_outliers('required_car_parking_spaces', method='std')
fix_outliers('total_of_special_requests', method='std')

normalize_columns = [ 
    'lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights',
    'adults', 'children', 'babies', 'previous_cancellations',
    'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list',
    'required_car_parking_spaces', 'total_of_special_requests'
]
standardize_columns = ['adr']

for col in normalize_columns:
    normalize(col)

for col in standardize_columns:
    standardize(col)

In [ ]:
df.describe()